In [27]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)


In [28]:
# all series
st_series = {'tos': 'tt0060028',
             'tas': 'tt0069637',
             'tng': 'tt0092455',
             'ds9': 'tt0106145',
             'voy': 'tt0112178',
             'ent': 'tt0244365',
             'dis': 'tt5171438',
             'sho': 'tt9059594',
             'pic': 'tt8806524',
             'lds': 'tt9184820',
             'pro': 'tt9795876',
             'snw': 'tt12327578',
             } 

## Series Request

In [30]:
# delete current database if exists
db_filename = "treklist.db"
if os.path.exists(db_filename):
    os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE IF EXISTS series")
conn.execute("CREATE TABLE series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster_url TEXT, poster BLOB, rated TEXT)")
for key in st_series.keys():
    res = omdb.imdbid(st_series[key])

    cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster_url, rated) VALUES (?, ?, ?, ?, ?, ?, ?) "
    curs.execute(cmd, [key, res['title'], res['imdb_id'], res['year'], res['total_seasons'], res['poster'], res['rated']])
    conn.commit()

df = pd.read_sql_query("SELECT * FROM series", conn)
df



,abb,title,imdb_id,year,total_seasons,poster_url,poster,rated
0,tos,Star Trek,tt0060028,1966–1969,3,https://m.media-amazon.com/images/M/MV5BNDRkMT...,None,TV-PG
1,tas,Star Trek: The Animated Series,tt0069637,1973–1975,2,https://m.media-amazon.com/images/M/MV5BMzI0Y2...,None,TV-Y7
2,tng,Star Trek: The Next Generation,tt0092455,1987–1994,7,https://m.media-amazon.com/images/M/MV5BOWFhYj...,None,TV-PG
3,ds9,Star Trek: Deep Space Nine,tt0106145,1993–1999,7,https://m.media-amazon.com/images/M/MV5BMDc3OG...,None,TV-PG
4,voy,Star Trek: Voyager,tt0112178,1995–2001,7,https://m.media-amazon.com/images/M/MV5BYWIwMT...,None,TV-PG
5,ent,Star Trek: Enterprise,tt0244365,2001–2005,4,https://m.media-amazon.com/images/M/MV5BODg3Zm...,None,TV-PG
6,dis,Star Trek: Discovery,tt5171438,2017–,5,https://m.media-amazon.com/images/M/MV5BNjg1NT...,None,TV-14
7,sho,Star Trek: Short Treks,tt9059594,2018–2020,2,https://m.media-amazon.com/images/M/MV5BNWY2NW...,None,TV-PG
8,pic,Star Trek: Picard,tt8806524,2020–,3,https://m.media-amazon.com/images/M/MV5BMmMzNT...,None,TV-MA
9,lds,Star Trek: Lower Decks,tt9184820,2020–,3,https://m.media-amazon.com/images/M/MV5BNjZkNG...,None,TV-14


## Episodes Request

In [31]:
keys = list(st_series.keys())
for key in keys: # temporary for testing
    conn.execute(f"CREATE TABLE IF NOT EXISTS {key} (title TEXT, rated TEXT, released DATE, season INTEGER, episode INTEGER, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, imdb_rating FLOAT, imdb_votes INTEGER, imdb_id TEXT)")

    tot_seasons = df['total_seasons'][df['abb'] == key].values[0]
    print('%s seasons %i' % (key, tot_seasons))
    for season in range(tot_seasons):# another temporary....
        season += 1 # account for zero index
        res = omdb.imdbid(st_series[key], season=season)
        for ep in res['episodes']:

            # check if it is already in the database
            df_check = pd.read_sql_query(f"SELECT * FROM {key} WHERE imdb_id = '{ep['imdb_id']}'", conn)
            ep_exists = True if len(df_check) > 0 else False
            if not ep_exists:
                ep_info = omdb.imdbid(ep['imdb_id'])
                cmd = f"INSERT INTO {key} (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep['released'], season, ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])
    conn.commit()

key tos seasons 3
key tas seasons 2
key tng seasons 7
key ds9 seasons 7
key voy seasons 7
key ent seasons 4
key dis seasons 5
key sho seasons 2
key pic seasons 3
key lds seasons 3
key pro seasons 1
key snw seasons 1


## Download/Insert Posters

In [32]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from series", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE series SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

In [46]:
# optional - read back a poster
curs.execute(f"SELECT * FROM series WHERE imdb_id = 'tt0060028'")
record = curs.fetchall()
idx = df.keys().to_list().index('poster')
image = Image.open(io.BytesIO(record[0][idx]))
image.show()

In [47]:
# download poster for individual series
for key in keys:
    print(key)
    df = pd.read_sql_query(f"SELECT * from {key}", conn)
    for i, row in df.iterrows():
        poster_url = row['poster_url']
        if poster_url != "N/A":
            # insert poster
            img = requests.get(row['poster_url'], stream=True)
            cmd = f"UPDATE {key} SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
            curs.execute(cmd, [sqlite3.Binary(img.content)])
            conn.commit()

tos
tas
tng
ds9
voy
ent
dis
sho
pic
lds
pro
snw


In [48]:
df = pd.read_sql_query(f"SELECT * from tos", conn)
df

,title,rated,released,season,episode,runtime,director,writer,actors,plot,poster_url,poster,imdb_rating,imdb_votes,imdb_id
0,The Man Trap,TV-PG,1966-09-08,1,1,50 min,Marc Daniels,"George Clayton Johnson, Gene Roddenberry","William Shatner, Leonard Nimoy, Jeanne Bal",Dr. McCoy discovers his old flame is not what ...,https://m.media-amazon.com/images/M/MV5BNDdiNG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.2,4853,tt0708469
1,Charlie X,TV-PG,1966-09-15,1,2,50 min,Lawrence Dobkin,"D.C. Fontana, Gene Roddenberry","William Shatner, Leonard Nimoy, Robert Walker Jr.",Captain Kirk must learn the limits to the powe...,https://m.media-amazon.com/images/M/MV5BZDdlY2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.1,4425,tt0708424
2,Where No Man Has Gone Before,TV-PG,1966-09-22,1,3,50 min,James Goldstone,"Samuel A. Peeples, Gene Roddenberry","William Shatner, Leonard Nimoy, Gary Lockwood",The flight recorder of the 200-year-old U.S.S....,https://m.media-amazon.com/images/M/MV5BOGIzMD...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,5431,tt0061027
3,The Naked Time,TV-PG,1966-09-29,1,4,50 min,Marc Daniels,"John D.F. Black, Gene Roddenberry","William Shatner, Leonard Nimoy, Stewart Moss",The crew is infected with a mysterious disease...,https://m.media-amazon.com/images/M/MV5BMjdjNj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.8,4278,tt0708473
4,The Enemy Within,TV-PG,1966-10-06,1,5,50 min,Leo Penn,"Richard Matheson, Gene Roddenberry","William Shatner, Leonard Nimoy, DeForest Kelley",A transporter malfunction splits Captain Kirk ...,https://m.media-amazon.com/images/M/MV5BZWQ2ZT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.6,4055,tt0708463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,The Way to Eden,TV-PG,1969-02-21,3,20,51 min,David Alexander,"Gene Roddenberry, Arthur Heinemann, D.C. Fontana","William Shatner, Leonard Nimoy, DeForest Kelley","A group of idealistic hippies, led by an irrat...",https://m.media-amazon.com/images/M/MV5BMDM1Mj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,5.4,2773,tt0708482
74,The Cloud Minders,TV-PG,1969-02-28,3,21,51 min,Jud Taylor,"Gene Roddenberry, Margaret Armen, David Gerrold","William Shatner, Leonard Nimoy, DeForest Kelley",Kirk and Spock are caught up in a revolution o...,https://m.media-amazon.com/images/M/MV5BMjU5Ym...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.0,2491,tt0708456
75,The Savage Curtain,TV-PG,1969-03-07,3,22,51 min,Herschel Daugherty,"Gene Roddenberry, Arthur Heinemann, Arthur H. ...","William Shatner, Leonard Nimoy, DeForest Kelley","Kirk, Spock, Abraham Lincoln and Vulcan legend...",https://m.media-amazon.com/images/M/MV5BNmZjM2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,2521,tt0708477
76,All Our Yesterdays,TV-PG,1969-03-14,3,23,50 min,Marvin J. Chomsky,"Gene Roddenberry, Jean Lisette Aroeste, Arthur...","William Shatner, Leonard Nimoy, DeForest Kelley","When Kirk, Spock and McCoy investigate the dis...",https://m.media-amazon.com/images/M/MV5BNDY3OT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,8.1,2766,tt0708415


## Movies Request

In [49]:
# all movies
st_movies = {'TMP':  'tt0079945',
             'TWOK': 'tt0084726',
             'TSFS': 'tt0088170',
             'TVH':  'tt0092007',
             'TFF':  'tt0098382',
             'TUC':  'tt0102975',
             'GEN':  'tt0111280',
             'FC':   'tt0117731',
             'INS':  'tt0120844',
             'NEM':  'tt0253754',
             'ST09': 'tt12126100',
             'STID': 'tt1408101',
             'STB':  'tt2660888',
             }

In [69]:
conn.execute("DROP TABLE IF EXISTS mov")
conn.execute(f"CREATE TABLE mov (abb TEXT, title TEXT, year TEXT, rated TEXT, released DATE, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, metascore INTEGER, imdb_rating FLOAT, imdb_votes INTEGER, box_office TEXT, imdb_id TEXT)")


for key in st_movies.keys():
    res = omdb.imdbid(st_movies[key])

    cmd = "INSERT INTO mov (abb, title, year, rated, released, runtime, director, writer, actors, plot, poster_url, metascore, imdb_rating, imdb_votes, imdb_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) "
    curs.execute(cmd, [key, res['title'], res['year'], res['rated'], res['released'], res['runtime'], res['director'], res['writer'], res['actors'], res['plot'], res['poster'], res['metascore'], res['imdb_rating'], res['imdb_votes'], res['imdb_id']])
    conn.commit()
    #if 'box_office' in res.keys():
    #    curs.execute(f"UPDATE mov SET box_office = ? WHERE imdb_id = '{res['imdb_id']}'", res['box_office'])
df = pd.read_sql_query("SELECT * FROM mov", conn)
df

,abb,title,year,rated,released,runtime,director,writer,actors,plot,poster_url,poster,metascore,imdb_rating,imdb_votes,box_office,imdb_id
0,TMP,Star Trek: The Motion Picture,1979,G,08 Dec 1979,132 min,Robert Wise,"Gene Roddenberry, Harold Livingston, Alan Dean...","William Shatner, Leonard Nimoy, DeForest Kelley",When an alien spacecraft of enormous power is ...,https://m.media-amazon.com/images/M/MV5BNjk1Zj...,None,48,6.4,"88,419",None,tt0079945
1,TWOK,Star Trek II: The Wrath of Khan,1982,PG,04 Jun 1982,113 min,Nicholas Meyer,"Gene Roddenberry, Harve Bennett, Jack B. Sowards","William Shatner, Leonard Nimoy, DeForest Kelley","With the assistance of the Enterprise crew, Ad...",https://m.media-amazon.com/images/M/MV5BNmZiZm...,None,67,7.7,"119,243",None,tt0084726
2,TSFS,Star Trek III: The Search for Spock,1984,PG,01 Jun 1984,105 min,Leonard Nimoy,"Gene Roddenberry, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley",Admiral Kirk and his bridge crew risk their ca...,https://m.media-amazon.com/images/M/MV5BMmFiMT...,None,56,6.6,"79,449",None,tt0088170
3,TVH,Star Trek IV: The Voyage Home,1986,PG,26 Nov 1986,119 min,Leonard Nimoy,"Gene Roddenberry, Leonard Nimoy, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley","To save Earth from an alien probe, Admiral Jam...",https://m.media-amazon.com/images/M/MV5BZmU3ZG...,None,71,7.3,"84,085",None,tt0092007
4,TFF,Star Trek V: The Final Frontier,1989,PG,09 Jun 1989,107 min,William Shatner,"Gene Roddenberry, William Shatner, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley",Captain Kirk and his crew must deal with Mr. S...,https://m.media-amazon.com/images/M/MV5BZmUzNj...,None,43,5.5,"59,389",None,tt0098382
5,TUC,Star Trek VI: The Undiscovered Country,1991,PG,06 Dec 1991,110 min,Nicholas Meyer,"Gene Roddenberry, Leonard Nimoy, Lawrence Konner","William Shatner, Leonard Nimoy, DeForest Kelley","On the eve of retirement, Kirk and McCoy are c...",https://m.media-amazon.com/images/M/MV5BYzJiZD...,None,65,7.2,"74,829",None,tt0102975
6,GEN,Star Trek: Generations,1994,PG,18 Nov 1994,118 min,David Carson,"Gene Roddenberry, Rick Berman, Ronald D. Moore","Patrick Stewart, William Shatner, Malcolm McDo...",With the help of long presumed dead Captain Ki...,https://m.media-amazon.com/images/M/MV5BOTk5Mj...,None,55,6.6,"80,920",None,tt0111280
7,FC,Star Trek: First Contact,1996,PG-13,22 Nov 1996,111 min,Jonathan Frakes,"Gene Roddenberry, Rick Berman, Brannon Braga","Patrick Stewart, Jonathan Frakes, Brent Spiner",The Borg travel back in time intent on prevent...,https://m.media-amazon.com/images/M/MV5BYzMzZm...,None,71,7.6,"123,981",None,tt0117731
8,INS,Star Trek: Insurrection,1998,PG,11 Dec 1998,103 min,Jonathan Frakes,"Gene Roddenberry, Rick Berman, Michael Piller","Patrick Stewart, Jonathan Frakes, Brent Spiner",When the crew of the Enterprise learn of a Fed...,https://m.media-amazon.com/images/M/MV5BNWEzZD...,None,64,6.4,"74,493",None,tt0120844
9,NEM,Star Trek: Nemesis,2002,PG-13,13 Dec 2002,116 min,Stuart Baird,"Gene Roddenberry, John Logan, Rick Berman","Patrick Stewart, Jonathan Frakes, Brent Spiner",The Enterprise is diverted to the Romulan home...,https://m.media-amazon.com/images/M/MV5BMjAxNj...,None,51,6.4,"78,345",None,tt0253754


## Download Movie Posters

In [53]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from mov", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE mov SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

,title,rated,released,season,episode,runtime,director,writer,actors,plot,poster_url,poster,imdb_rating,imdb_votes,imdb_id
0,The Man Trap,TV-PG,1966-09-08,1,1,50 min,Marc Daniels,"George Clayton Johnson, Gene Roddenberry","William Shatner, Leonard Nimoy, Jeanne Bal",Dr. McCoy discovers his old flame is not what ...,https://m.media-amazon.com/images/M/MV5BNDdiNG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.2,4853,tt0708469
1,Charlie X,TV-PG,1966-09-15,1,2,50 min,Lawrence Dobkin,"D.C. Fontana, Gene Roddenberry","William Shatner, Leonard Nimoy, Robert Walker Jr.",Captain Kirk must learn the limits to the powe...,https://m.media-amazon.com/images/M/MV5BZDdlY2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.1,4425,tt0708424
2,Where No Man Has Gone Before,TV-PG,1966-09-22,1,3,50 min,James Goldstone,"Samuel A. Peeples, Gene Roddenberry","William Shatner, Leonard Nimoy, Gary Lockwood",The flight recorder of the 200-year-old U.S.S....,https://m.media-amazon.com/images/M/MV5BOGIzMD...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,5431,tt0061027
3,The Naked Time,TV-PG,1966-09-29,1,4,50 min,Marc Daniels,"John D.F. Black, Gene Roddenberry","William Shatner, Leonard Nimoy, Stewart Moss",The crew is infected with a mysterious disease...,https://m.media-amazon.com/images/M/MV5BMjdjNj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.8,4278,tt0708473
4,The Enemy Within,TV-PG,1966-10-06,1,5,50 min,Leo Penn,"Richard Matheson, Gene Roddenberry","William Shatner, Leonard Nimoy, DeForest Kelley",A transporter malfunction splits Captain Kirk ...,https://m.media-amazon.com/images/M/MV5BZWQ2ZT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.6,4055,tt0708463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,The Way to Eden,TV-PG,1969-02-21,3,20,51 min,David Alexander,"Gene Roddenberry, Arthur Heinemann, D.C. Fontana","William Shatner, Leonard Nimoy, DeForest Kelley","A group of idealistic hippies, led by an irrat...",https://m.media-amazon.com/images/M/MV5BMDM1Mj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,5.4,2773,tt0708482
74,The Cloud Minders,TV-PG,1969-02-28,3,21,51 min,Jud Taylor,"Gene Roddenberry, Margaret Armen, David Gerrold","William Shatner, Leonard Nimoy, DeForest Kelley",Kirk and Spock are caught up in a revolution o...,https://m.media-amazon.com/images/M/MV5BMjU5Ym...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.0,2491,tt0708456
75,The Savage Curtain,TV-PG,1969-03-07,3,22,51 min,Herschel Daugherty,"Gene Roddenberry, Arthur Heinemann, Arthur H. ...","William Shatner, Leonard Nimoy, DeForest Kelley","Kirk, Spock, Abraham Lincoln and Vulcan legend...",https://m.media-amazon.com/images/M/MV5BNmZjM2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,2521,tt0708477
76,All Our Yesterdays,TV-PG,1969-03-14,3,23,50 min,Marvin J. Chomsky,"Gene Roddenberry, Jean Lisette Aroeste, Arthur...","William Shatner, Leonard Nimoy, DeForest Kelley","When Kirk, Spock and McCoy investigate the dis...",https://m.media-amazon.com/images/M/MV5BNDY3OT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,8.1,2766,tt0708415
